In [1]:
# load Chinese stopwords
with open('chinese_stopwords.txt','r',encoding='utf-8') as file: 
    stopwords = [i[:-1] for i in file.readlines()]
print(stopwords)

['，', '的', '。', '、', '在', '了', '是', '\u3000', '“', '”', '和', '年', '月', '：', '也', '）', '为', '（', '有', '%', '日', '将', '中', '-', '到', '与', '对', ':', '\xa0', '上', '都', '等', '不', '他', '》', '《', '就', '但', '我', '而', '这', '会', '并', '；', '被', '后', '人', '从', '还', '1', '3', '6', '以', '新', '说', '7', '2', '要', '5', '？', '更', '于', '个', '10', '大', '时', '4', '多', '/', '让', '其', ')', '(', '很', '及', '下', '', '能', '—', '或', '该', '她', '比', '8', '元', '12', '已', '向', '做', '来', '前', '由', '好', '.', '称', '给', '最', '11', '·', '据', '着', '又', '至', '9', '20', '！', '[', ']', '去', '可', '把', '则', '', '一', '地', '高', '吗', '30', '所', '分', '较', '内', '第', '里', '占', '过', '15', '曾', '"', '再', '人民日报', '新闻网', '它', '况', '而且', '而是', '而外', '而言', '而已', '尔后', '反过来', '反过来说', '反之', '非但', '非徒', '否则', '嘎', '嘎登', '该', '赶', '个', '各', '各个', '各位', '各种', '各自', '给', '根据', '跟', '故', '故此', '固然', '关于', '管', '归', '果然', '果真', '过', '哈', '哈哈', '呵', '和', '何', '何处', '何况', '何时', '嘿', '哼', '哼唷', '呼哧', '乎', '哗', '还是', '还有', '换句话说', '换言之', '或', '或是', '或

In [2]:
# load data
import pandas as pd
news = pd.read_csv('sqlResult.csv',encoding='gb18030')
print(news.shape)
print(news.head())

(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...   
3    这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [3]:
# missing data
news = news.dropna(subset=['content'])
print(news.shape)

(87054, 7)


In [4]:
# split words
import jieba
def split_text(text):
    text = text.replace(' ','').replace('/n','') #data cleaning
    text2 =jieba.cut(text.strip())
    # drop stopwords
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

In [5]:
print(news.iloc[0].content)
print(split_text(news.iloc[0].content))

Building prefix dict from the default dictionary ...


此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。



Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.929 seconds.
Prefix dict has been built successfully.


此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 
 MIUI8 去年 发布 距今已有 一年 有余 更新换代 
 当然 MIUI9 确切 信息 等待 官方消息


In [6]:
# split works for all the text info
corpus = list(map(split_text,[str(i) for i in news.content]))
print(corpus[0])
print(len(corpus))

此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 
 MIUI8 去年 发布 距今已有 一年 有余 更新换代 
 当然 MIUI9 确切 信息 等待 官方消息
87054


In [14]:
import pickle,os
if not os.path.exists('corpus.pkl'):
     # save for future usage
    with open('corpus.pkl','wb') as file: 
        pickle.dump(corpus,file)
    
else:
    with open('corpus.pkl','rb') as file:
        corpus = pickle.load(file)  
    
    
# compute corpus matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
countvectorizer = CountVectorizer(encoding ='gb18030',min_df=0.015)
tfidftransformer = TfidfTransformer()
# tf, then idf =>TFIDF
countvector = countvectorizer.fit_transform(corpus)
tfidf = tfidftransformer.fit_transform(countvector)


In [17]:
# label if its our own news
label = list(map(lambda source:1 if '新华' in str(source) else 0, news.source))
# split and train
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import numpy as np

x_train, x_test, y_train, y_test = train_test_split(tfidf.toarray(),label,test_size=0.3)
clf = MultinomialNB()
clf.fit(x_train,y_train)
y_predict = clf.predict(x_test)
# use model to test plagiarism and forcast styles
# prediction=1, xinhua style
# lable=1, real xinhua 
prediction = clf.predict(tfidf.toarray())
labels =np.array(label)
# compare news index: two columns, prediction and labels=reality 
compare_news_index =pd.DataFrame({'prediction':prediction, 'labels':labels})

copy_news_index = compare_news_index[(compare_news_index['prediction']==1) & (compare_news_index['labels']==0)].index
# xinhua news
xinhuashe_news_index = compare_news_index[(compare_news_index['labels']==1)].index
print('possible plagiarism',len(copy_news_index))

possible plagiarism 2809


In [22]:
#use kmeans for clustering
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans

normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())
# 25 is the number of buskets
kmeans = KMeans(n_clusters=10)
k_labels = kmeans.fit_predict(scaled_array)


In [28]:
id_class={index:class_ for index, class_ in enumerate(k_labels)}
from collections import defaultdict
class_id = defaultdict(set)
for index, class_ in id_class.items():
    # xinhua class_id only
    if index in xinhuashe_news_index.tolist():
        class_id[class_].add(index)
#output 
#search similar text
from sklearn.metrics.pairwise import cosine_similarity
def find_similar_text(cpindex,top=10):
    dist_dict = {i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]} 
    # sort from smallest to largest
    return sorted(dist-dict.items(), key=lambda x:x[1][0],reverse=True)[:top]

# test, select an article number from ok 89k articles, cpindex is the subscript
cpindex =3352

similar_list = find_similar_text(cpindex)
print(similar_list)
print('possible plagiarism articles:\n',news_iloc[cpindex].content)
# find a similar 
similar2 = simiar_list[0][0]
print('similar original:\n', news.iloc[similar2].content)

NameError: name 'dist' is not defined